In [2]:
#surprise를 이용한 추천 시스템
# https://github.com/wikibook/ml-definitive-guide

1.1.1


In [ ]:
import surprise 

print(surprise.__version__)

In [3]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split 

In [9]:
data = Dataset.load_builtin('ml-100k')
trainset,testset = train_test_split(data, test_size=0.25, random_state=0)

In [10]:
#algo=SVD() 알고리즘 객체 생성

algo = SVD()
algo.fit(trainset)

In [11]:
predictions = algo.test(testset)
print('prediction type:', type(predictions), ' size:', len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type: <class 'list'>  size: 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.780726169551054, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.7747850644210454, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.031778835490394, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.669368828442081, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.3348097547397524, details={'was_impossible': False})]

In [15]:
[(pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('120', '282', 3.780726169551054),
 ('882', '291', 3.7747850644210454),
 ('535', '507', 4.031778835490394)]

In [16]:
#사용자아이디, 아이템아이디는 문자열로 입려
uid = str(196)
iid = str(302)
pred = algo.predict(uid,iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.09   {'was_impossible': False}


In [17]:
accuracy.rmse(predictions)

RMSE: 0.9483


0.9483273370924832

In [18]:
import pandas as pd 

ratings = pd.read_csv('ratings.csv')

#인덱스와 헤드 제거한 파일 생성 -> ratings_noh로 파일 생성
ratings.to_csv('ratings_noh.csv', index=False, header=False)

In [20]:
from surprise import Reader

reader = Reader(line_format = 'user item rating timestamp',sep=',', rating_scale=(0.5,5))
data = Dataset.load_from_file('ratings_noh.csv',reader=reader) #무비렌즈데이터 형식이 아닌 다른 os 파일의 경우, reader 클래스 설정

In [25]:
trainset, testset = train_test_split(data, test_size=0.25, random_state=0)
algo = SVD(n_factors=50, random_state=0)

#테스트데이터로 평점예측 후 RMSE 평가
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

# 교차검증과 하이퍼파라미터튜닝

In [28]:
from surprise.model_selection import cross_validate

#판다스 DataFrame에서 surprise 데이터 세트로 로딩
ratings = pd.read_csv('ratings.csv')
reader=Reader(rating_scale=(0.5,5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']],reader)

algo = SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE','MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8630  0.8751  0.8821  0.8767  0.8720  0.8738  0.0063  
MAE (testset)     0.6637  0.6698  0.6769  0.6738  0.6718  0.6712  0.0044  
Fit time          3.79    3.85    3.83    3.78    3.78    3.81    0.03    
Test time         0.15    0.10    0.09    0.14    0.14    0.13    0.03    


{'test_rmse': array([0.86301574, 0.87505265, 0.88208908, 0.87667043, 0.87197541]),
 'test_mae': array([0.6636655 , 0.66983096, 0.67688748, 0.67378355, 0.67180694]),
 'fit_time': (3.793497085571289,
  3.8471105098724365,
  3.834508180618286,
  3.778911590576172,
  3.7848143577575684),
 'test_time': (0.1511247158050537,
  0.09530305862426758,
  0.09267425537109375,
  0.14461064338684082,
  0.14367938041687012)}

In [30]:
from surprise.model_selection import GridSearchCV

#최적화할 파라미터를 딕셔너리 형태로 지정
param_grid = {'n_epochs':[20,40,60], 'n_factors': [50,100,200]}

#CV를 3개 폴트 세트로 지정-> 성능 평가: rmse,mse로 수행하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

#최고 RMSE 평가 점수와 하이퍼파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8775210084584106
{'n_epochs': 20, 'n_factors': 50}


# surprise를 이용한 개인화 영화 추천 시스템 구축

In [32]:
#데이터세트 전체를 학습 데이터로 사용 :DatasetAutoFolds 클래스 이용

from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format = 'user item rating timestamp', sep=',', rating_scale=(0.5,5))
data_folds = DatasetAutoFolds(ratings_file='ratings_noh.csv', reader=reader)

#전체 데이터를 학습 데이터로 생성
trainset = data_folds.build_full_trainset()

In [33]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

In [34]:
#특정사용자(userId=9)가 아직 평점을 매기지 않은 영화(movieId 42)로 선정 뒤, 예측 평점 계산
#영화에 대한 상세 속성 정보 DataFrame 로딩
movies = pd.read_csv('movies.csv')

#suerId=9의 movieId데이터를 추출해 movieId=42 데이터가 있는지 확인
movieIds = ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds==42].count() ==0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')
    
print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [35]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


In [36]:
#사용자가 평점을 매기지 않은 전체 영화를 추출한 뒤 예측 평점순으로 영화 추천

def get_unseen_surprise(ratings, movies, userId):
    seen_movies = ratings[ratings['userId']==userId]['movieId'].tolist() #사용자가 평점을 매긴 모든 영화를 리스트로 생성
    
    total_movies = movies['movieId'].tolist() #모든 영화를 리스트로 생성
    
    #모든 영화 중 이미 평점을 매긴 영화를 제외 후 리스트 생성
    unseen_movies = [movie for movie in total_movies if movie not in seen_movies] 
    print('평점 매긴 영화 수:', len(seen_movies), '추천 대상 영화 수:', len(unseen_movies),
         '전체 영화 수:',len(total_movies))
    
    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화 수: 46 추천 대상 영화 수: 9696 전체 영화 수: 9742


In [39]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies] #predict()를 평점이 없는 여화에 반복 후 list 객체로 저장
    
    def sortkey_est(pred):
        return pred.est
    
    #sortkey_est 반환값의 내림차순으로 정렬 후 top_n 개의 최상위 값 추출
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions = predictions[:top_n]
    
    #top_n으로 추출된 영화정보/아이디/추천예상평점/제목 추출
    top_movie_ids = [int(pred.iid) for pred in top_predictions]
    top_movie_rating = [pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    
    top_movie_preds = [(id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]
    
    return top_movie_preds

In [40]:
unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n =10)

print('#### Top-10 추천 영화 리스트 ###')
for top_movie in top_movie_preds:
    print(top_movie[1],":", top_movie[2])

평점 매긴 영화 수: 46 추천 대상 영화 수: 9696 전체 영화 수: 9742
#### Top-10 추천 영화 리스트 ###
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122658
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957
